In [1]:
from pykrx import stock

In [33]:
def fetch_market_data_range(start_date: str, end_date: str):
    # pykrx의 "모든 종목" 일자별 조회 API는 일괄 반환함
    
    price_df = stock.get_market_ohlcv_by_date(fromdate=start_date, todate=end_date, ticker="005930", adjusted=True)
    fund_df = stock.get_market_fundamental_by_date(fromdate=start_date, todate=end_date, ticker="005930")
    cap_df = stock.get_market_cap_by_date(fromdate=start_date, todate=end_date, ticker="005930")
    foreign_df = stock.get_exhaustion_rates_of_foreign_investment_by_date(fromdate=start_date, todate=end_date, ticker="005930")

        # 인덱스 리셋 → merge를 위해 date, code 컬럼으로 만들기
    def fix_index(df, value_cols):
        df = df.copy()
        df.reset_index(inplace=True)
        df = df.rename(columns={"날짜": "date"})
        df = df[value_cols + ['date']]
        return df
    
    print(price_df.head(2))
    print(fund_df.head(2))
    print(cap_df.head(2))
    print(foreign_df.head(2))

    price_df = fix_index(price_df, ['시가', '고가', '저가', '종가', '거래량', '등락률'])
    fund_df = fix_index(fund_df, ['BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'])
    cap_df = fix_index(cap_df, ['시가총액', '거래대금', '상장주식수'])  # 거래량은 중복되므로 제거 가능
    foreign_df = fix_index(foreign_df, ['보유수량', '지분율', '한도수량', '한도소진률'])

    merged = price_df.merge(fund_df, on=['date'], how='left')\
                     .merge(cap_df, on=['date'], how='left')\
                     .merge(foreign_df, on=['date'], how='left')
    print(merged.head(2))

In [34]:
fetch_market_data_range("20230101", "20240101")

               시가     고가     저가     종가       거래량       등락률
날짜                                                        
2023-01-02  55500  56100  55200  55500  10031448  0.361664
2023-01-03  55400  56000  54500  55400  13547030 -0.180180
              BPS   PER   PBR   EPS   DIV   DPS
날짜                                             
2023-01-02  43611  9.61  1.27  5777  2.60  1444
2023-01-03  43611  9.59  1.27  5777  2.61  1444
                       시가총액       거래량          거래대금       상장주식수
날짜                                                             
2023-01-02  331322931525000  10031448  558433491400  5969782550
2023-01-03  330725953270000  13547030  747898872200  5969782550
                 상장주식수        보유수량       지분율        한도수량     한도소진률
날짜                                                                
2023-01-02  5969782550  2965339762  49.65625  5969782550  49.65625
2023-01-03  5969782550  2964905145  49.65625  5969782550  49.65625
      시가     고가     저가     종가  ...        보유수량  